# Combining Datasets: Merge and Join
One essential feature offered by Pandas is its high-performance, in-memory join and
merge operations. If you have ever worked with databases, you should be familiar
with this type of data interaction. The main interface for this is the pd.merge function.

## Categories of Joins
The <b>pd.merge()</b> function implements a number of types of joins: the<i> one-to-one</i>,
<i>many-to-one</i>, and <i>many-to-many</i> joins. All three types of joins are accessed via an
identical call to the <b>pd.merge()</b> interface; the type of join performed depends on the
form of the input data.

### One-to-one joins
Perhaps the simplest type of merge expression is the <i>one-to-one</i> join, which is in
many ways very similar to the column-wise concatenation. 

As a concrete example, consider the following
two DataFrames, which contain information on several employees in a company:

In [3]:
import numpy as np 
import pandas as pd 

In [4]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'], 'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],'hire_date': [2004, 2008, 2012, 2014]})
print(df1); print(df2)

employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


To combine this information into a single DataFrame, we can use the pd.merge()
function:

In [11]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


The pd.merge() function recognizes that each DataFrame has an “employee” column,
and automatically joins using this column as a key. The result of the merge is a new
DataFrame that combines the information from the two inputs. Notice that the order
of entries in each column is not necessarily maintained: in this case, the order of the
“employee” column differs between df1 and df2, and the pd.merge() function correctly
accounts for this. Additionally, keep in mind that the merge in general discards
the index, except in the special case of merges by index i.e., “The left_index and
right_index"

### Many-to-one joins
Many-to-one joins are joins in which one of the two key columns contains duplicate
entries. For the many-to-one case, the resulting DataFrame will preserve those duplicate
entries as appropriate. 

Consider the following example of a many-to-one join:

In [12]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3); print(df4); print(pd.merge(df3, df4))

employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve


The resulting DataFrame has an additional column with the “supervisor” information,
where the information is repeated in one or more locations as required by the inputs.

### Many-to-many joins
Many-to-many joins are a bit confusing conceptually, but are nevertheless well
defined. If the key column in both the left and right array contains duplicates, then
the result is a many-to-many merge. 

This will be perhaps most clear with a concrete
example. Consider the following, where we have a DataFrame showing one or more
skills associated with a particular group.
By performing a many-to-many join, we can recover the skills associated with any
individual person:

In [13]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting','Engineering', 'Engineering', 'HR', 'HR'],
'skills': ['math', 'spreadsheets', 'coding', 'linux','spreadsheets', 'organization']})
print(df1); print(df5); print(pd.merge(df1, df5))

employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization
  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


## Specification of the Merge Key
We’ve already seen the default behavior of <mark><b>pd.merge()</b></mark>: it looks for one or more
matching column names between the two inputs, and uses this as the key. However,
often the column names will not match so nicely, and pd.merge() provides a variety
of options for handling this.

### The on keyword
Most simply, you can explicitly specify the name of the key column using the on keyword,
which takes a column name or a list of column names:

In [16]:
# The on keyword
# This option works only if both the left and right DataFrames have the specified column name.
print(df1); print(df2); print(pd.merge(df1, df2, on='employee'))

employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


### The left_on and right_on keywords
At times you may wish to merge two datasets with different column names; for example,
we may have a dataset in which the employee name is labeled as “name” rather
than “employee”. In this case, we can use the left_on and right_on keywords to
specify the two column names:

In [19]:
# the left_on and right_on keywords
# The result has a redundant column that we can drop if desired—for example, by using the drop() method of DataFrames:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
'salary': [70000, 80000, 120000, 90000]})
print(df1); print(df3);print(pd.merge(df1, df3, left_on="employee", right_on="name"))

employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000


In [20]:
# The result has a redundant column that we can drop if desired—for example, by using the drop() method of DataFrames:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### The left_index and right_index keywords
Sometimes, rather than merging on a column, you would instead like to merge on an
index. 

For example, your data might look like this:

In [22]:
# the left_index and right_index keywords
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a); print(df2a)
# You can use the index as the key for merging by specifying the left_index and/or right_index flags in pd.merge():
print(pd.merge(df1a, df2a, left_index=True, right_index=True))

group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


For convenience, DataFrames implement the join() method, which performs a
merge that defaults to joining on indices:

In [25]:
# For convenience, DataFrames implement the join() method, which performs a merge that defaults to joining on indices:
print(df1a); print(df2a); print(df1a.join(df2a))

group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


If you’d like to mix indices and columns, you can combine left_index with right_on
or left_on with right_index to get the desired behavior:

In [26]:
print(df1a); print(df3); print(pd.merge(df1a, df3, left_index=True, right_on='name'))

group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000


## Specifying Set Arithmetic for Joins
In all the preceding examples we have glossed over one important consideration in
performing a join: the type of set arithmetic used in the join. This comes up when a
value appears in one key column but not the other. 

Consider this example:

In [27]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
'food': ['fish', 'beans', 'bread']}, columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
'drink': ['wine', 'beer']}, columns=['name', 'drink'])
print(df6); print(df7); print(pd.merge(df6, df7))

name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
   name   food drink
0  Mary  bread  wine


Here we have merged two datasets that have only a single “name” entry in common:
Mary. By default, the result contains the intersection of the two sets of inputs; this is
what is known as an inner join. We can specify this explicitly using the how keyword,
which defaults to <mark><b>'inner'</b></mark>:

In [29]:
# inner join
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Other options for the how keyword are <i>'outer'</i>,<i> 'left'</i>, and<i> 'right'</i>. An outer join
returns a join over the union of the input columns, and fills in all missing values with
NAs:

In [31]:
# outer join
print(df6); print(df7); print(pd.merge(df6, df7, how='outer'))

name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer


The <i>left join</i> and<i> right join</i> return join over the left entries and right entries, respectively.

For example:

In [34]:
# left join
# The output rows now correspond to the entries in the left input.
print(df6); print(df7); print(pd.merge(df6, df7, how='left'))

name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine


In [36]:
# right join
print(df6);print(df7); print(pd.merge(df6, df7, how='right'))

name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer


## Overlapping Column Names: The suffixes Keyword
Finally, you may end up in a case where your two input DataFrames have conflicting
column names. 

Consider this example:

In [37]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
'rank': [3, 1, 4, 2]})
print(df8); print(df9); print(pd.merge(df8, df9, on="name"))

name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


Because the output would have two conflicting column names, the merge function
automatically appends a suffix _x or _y to make the output columns unique. If these
defaults are inappropriate, it is possible to specify a custom suffix using the suffixes
keyword:

In [39]:
# custom suffixes
print(df8); print(df9); print(pd.merge(df8, df9, on="name", suffixes=["_L", "_R"]))

name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


## Example: US States Data
Merge and join operations come up most often when one is combining data from different
sources. Here we will consider an example of some data about US states and
their populations. 

The data files can be found at http://github.com/jakevdp/data-USstates/:

In [40]:
# Let’s take a look at the three datasets, using the Pandas read_csv() function:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

print(pop.head()); print(areas.head()); print(abbrevs.head())

state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


Given this information, say we want to compute a relatively straightforward result:
rank US states and territories by their 2010 population density. We clearly have the
data here to find this result, but we’ll have to combine the datasets to get it.

We’ll start with a many-to-one merge that will give us the full state name within the
population DataFrame. We want to merge based on the state/region column of pop,
and the abbreviation column of abbrevs. We’ll use how='outer' to make sure no
data is thrown away due to mismatched labels.

In [99]:
merge_1 = pd.merge(pop, abbrevs, how = 'outer', left_on='state/region', right_on='abbreviation')
merge_1 = merge_1.drop('abbreviation', axis = 1) # drop duplicate info
merge_1.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Let’s double-check whether there were any mismatches here, which we can do by
looking for rows with nulls:

In [100]:
merge_1.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Some of the population info is null; let’s figure out which these are!

In [101]:
merge_1[merge_1['state'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


It appears that all the null population values are from Puerto Rico prior to the year
2000; this is likely due to this data not being available from the original source.

More importantly, we see also that some of the new state entries are also null, which
means that there was no corresponding entry in the abbrevs key! Let’s figure out
which regions lack this match:

In [104]:
merge_1.loc[merge_1['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

We can quickly infer the issue: our population data includes entries for Puerto Rico
(PR) and the United States as a whole (USA), while these entries do not appear in the
state abbreviation key. We can fix these quickly by filling in appropriate entries:

In [106]:
merge_1.loc[merge_1['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merge_1.loc[merge_1['state/region'] == 'USA', 'state'] = 'United States'
merge_1.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

No more nulls in the state column: we’re all set!

Now we can merge the result with the area data using a similar procedure. Examining
our results, we will want to join on the state column in both:

In [107]:
merge_2 = pd.merge(merge_1, areas, on = 'state', how='left')
merge_2.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [108]:
# Again, let’s check for nulls to see if there were any mismatches:
merge_2.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

There are nulls in the area column; we can take a look to see which regions were
ignored here:

In [109]:
merge_2['state'][merge_2['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

We see that our areas DataFrame does not contain the area of the United States as a
whole. We could insert the appropriate value (using the sum of all state areas, for
instance), but in this case we’ll just drop the null values because the population density
of the entire United States is not relevant to our current discussion:

In [110]:
merge_2.dropna(inplace=True)
merge_2.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Now we have all the data we need. To answer the question of interest, let’s first select
the portion of the data corresponding with the year 2000, and the total population.
We’ll use the query() function to do this quickly (this requires the numexpr package
to be installed;

In [111]:
data2010 = merge_2.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Now let’s compute the population density and display it in order. We’ll start by reindexing
our data on the state, and then compute the result:

In [112]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of
their 2010 population density, in residents per square mile. We can see that by far the
densest region in this dataset is Washington, DC (i.e., the District of Columbia);
among states, the densest is New Jersey.

We can also check the end of the list:

In [114]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

We see that the least dense state, by far, is Alaska, averaging slightly over one resident
per square mile.